In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import re

# 데이터 불러오기
df = pd.read_csv('/content/drugLibTrain_raw_new.tsv', sep="\t", encoding='latin1')

# 필요한 열 선택 및 결측치 제거
df = df[['urlDrugName', 'benefitsReview', 'sideEffectsReview', 'commentsReview', 'condition']]
df = df.dropna()

# 모든 리뷰 합치기
df['all_reviews'] = df['benefitsReview'] + ' ' + df['sideEffectsReview'] + ' ' + df['commentsReview']

# 텍스트 전처리
df['all_reviews'] = df['all_reviews'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
df['all_reviews'] = df['all_reviews'].str.lower()
df['condition'] = df['condition'].str.lower()

# 토큰화 및 불용어 제거
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['all_reviews'])
review_sequences = tokenizer.texts_to_sequences(df['all_reviews'])
max_length = max([len(seq) for seq in review_sequences])
df_review_padded = pad_sequences(review_sequences, maxlen=max_length, padding='post')
df['all_reviews'] = list(df_review_padded)

# 데이터 분할
X = df[['all_reviews']]
y = df[['condition']]
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# NumPy 배열을 텐서로 변환
X_train = np.array(X_train['all_reviews'].tolist())
X_test = np.array(X_test['all_reviews'].tolist())
# 모델 생성
model = Sequential()
embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
# 출력 레이어의 노드 수를 클래스의 수로 변경
num_classes = len(np.unique(y_train))
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-